In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import allmusicUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from ioUtils import getFile, saveFile
from fileUtils import getFileBasics, getBasename
from webUtils import getWebData, getHTML, getURL
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-28 16:56:15.973138


In [2]:
%load_ext autoreload
%autoreload
disc   = discogs('allmusic')
dutils = allmusicUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Warning! Saved Discog Directory None is not Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Users/tgadfort/Music/Discog/artists-allmusic exists
Creating /Users/tgadfort/Music/Discog/albums-allmusic
Creating /Users/tgadfort/Music/Discog/albums-allmusic
/Users/tgadfort/Music/Discog/artists-allmusic-db exists
/Users/tgadfort/Music/Discog/albums-allmusic-db exists
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata exists
/Users/tgadfort/Music/Discog/albums-allmusic-db/metadata exists
Creating /Users/tgadfort/Music/Discog/diagnostic
Creating /Users/tgadfort/Music/Discog/diagnostic
/Users/tgadfort/Music/Discog/db exists


# Artists From Search

In [4]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
arts.searchDiscogForArtist("Dababy")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


===================== Searching For Dababy =====================
Downloading: https://www.discogs.com/search/?q=Dababy&limit=250&type=artist
Found 2 artists
2 / 2 	: 7 	 https://www.discogs.com/artist/6619834-Dababy?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/34/6619834.p (force=False)
  --> This file is 18.5kB.
Done. Sleeping for 2 seconds


# Albums From Known People

In [ ]:
data = getFile("/Volumes/Music/MusicDB/toget.p")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

_ = clock("Last Run")

In [ ]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
toget={}
known={}

for ia,artistName in enumerate(data):
    #artistID = artistName
    artistID = None
    if artistID is None:
        retval = getMusicData('Name', artistName)
        if retval is not None:
            artistID = retval.index.values[0]
    modValue  = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
    if toget.get(modValue) is None:
        toget[modValue] = {}
    toget[modValue][artistID] = True
        
    if ia % 100 == 0:
        print(ia,'/',len(data))

In [ ]:
for modVal in toget.keys():
    artistIDs = toget[modVal]
    print(modVal,'\t',len(artistIDs))
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID in artistIDs:
        print(' ',artistID)
        artistData = dbdata[artistID]
        albs.downloadAlbumFromArtistData(artistID=artistID, artistData=artistData, mediaTypes=None)

# Albums From All Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=1, mediaTypes=None, rand=True)

In [8]:
def getURLData(url, debug=True):    
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    if debug:
        print("Now Downloading: {0}".format(url))

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need
    
    return data, response

In [4]:
myMusicMap = getFile("/Users/tgadfort/Documents/code/music/musicDiscogsMap.p")

In [6]:
allMusicIDMap = {}
savename = "collections/allMusicIDMap.p"
allMusicIDMap = getFile(savename)
len(allMusicIDMap)

38218

In [69]:
from json import loads
baseURL = "https://www.allmusic.com/search/artists/"
for i,(artistName,artistVals) in enumerate(myMusicMap.items()):
    if i < 3871:
        continue
    print(i,'/',len(myMusicMap),'  \t',artistName)
    url     = urllib.parse.urljoin(baseURL, quote(artistName))

    data, response = getURLData(url)
    if response.code == 200:
        bsdata = getHTML(data)
    else:
        continue
        
    infos = bsdata.findAll("div", {"class": "info"})
    for info in infos:
        for name in info.findAll("div", {"class": "name"}):
            ref = name.find("a")
            if ref is None:
                continue
            href     = ref.attrs['href']
            artistID = loads(ref.attrs['data-tooltip'])['id']
            name     = ref.text
            if allMusicIDMap.get(artistID) is None:
                allMusicIDMap[artistID] = [name, href]
                print("\t{0: <70}{1: <20}".format(name,artistID))
                
    print("  -->",len(allMusicIDMap))
    sleep(2)
    if i % 50 == 0:
        savename = "collections/allMusicIDMap.p"
        print("Saving {0} AllMusic IDs to {1}".format(len(allMusicIDMap), savename))
        saveFile(idata=allMusicIDMap, ifile=savename, debug=True)

3871 / 4692   	 America
Now Downloading: https://www.allmusic.com/search/artists/America
  --> 45292
3872 / 4692   	 Amoeba
Now Downloading: https://www.allmusic.com/search/artists/Amoeba
	Amoeba                                                                MN0000749007        
	Amoeba Assassin                                                       MN0000020663        
	Amoeba Crunch                                                         MN0001564262        
	The Amoeba People                                                     MN0002861664        
	The Amoeba Collective                                                 MN0003061876        
	Amoeba Teen                                                           MN0003770497        
	Ola Amoeba Death                                                      MN0002831752        
	The Amboy Dukes                                                       MN0000046120        
	Amba Shepherd                                                         MN000

3879 / 4692   	 Asia
Now Downloading: https://www.allmusic.com/search/artists/Asia
	Asia                                                                  MN0003443552        
	Asia                                                                  MN0002919005        
	Lyrics Born                                                           MN0000207640        
	Asia Argento                                                          MN0000991737        
	Asia Born                                                             MN0001905757        
	Asia Minor                                                            MN0002147687        
	Asia Bryant                                                           MN0002584099        
	Asia Whiteacre                                                        MN0003407625        
	Daniel Asia                                                           MN0001843985        
	Asia Johnson                                                          MN0000333304      

	Avant                                                                 MN0000062412        
	Avant                                                                 MN0002739819        
	Avant Garde                                                           MN0000912383        
	Avant Duel                                                            MN0002982786        
	Avant Strangel                                                        MN0000491983        
	Avant Garde                                                           MN0001445043        
	The Avant Garden                                                      MN0002002487        
	Avant Garde                                                           MN0002104773        
	Avant Guards                                                          MN0002122497        
	Avant Garage                                                          MN0002286611        
	Avant Gardes                                                          MN0002313

3890 / 4692   	 Beltaine
Now Downloading: https://www.allmusic.com/search/artists/Beltaine
	Beltaine                                                              MN0002905648        
	Beltaine                                                              MN0001346192        
	The Bluetones                                                         MN0000039342        
	Ivor Bolton                                                           MN0000793025        
	Aldo Baldin                                                           MN0001880630        
	The Bleedin Bleedins                                                  MN0002052619        
	Belladonna                                                            MN0000152504        
	Beltones                                                              MN0002635540        
	Bluedawn                                                              MN0002690591        
	Boldan                                                                MN00037462

3896 / 4692   	 Blackhouse
Now Downloading: https://www.allmusic.com/search/artists/Blackhouse
	Blackhouse                                                            MN0000097300        
	The Blackhouse                                                        MN0003048262        
	Blackhouse Productions                                                MN0001285070        
	Blackhouse Project                                                    MN0002049824        
	Helen Blackhouse                                                      MN0001216848        
	Wesley Blackhouse                                                     MN0002469116        
	Lamontt "Spec" Blackhouse                                             MN0003407964        
	Blockhouses                                                           MN0003785872        
	Blockhouse Gang                                                       MN0000713453        
	Blockhouse Bay                                                        MN0003

...

4664 / 4692   	 Qantice
Now Downloading: https://www.allmusic.com/search/artists/Qantice
	Candice Glover                                                        MN0003112029        
	Candace Smith                                                         MN0001612940        
	Paul Kuentz                                                           MN0001281778        
	Candice Night                                                         MN0000718738        
	Candice Boyd                                                          MN0002595516        
	Candice Pillay                                                        MN0002837466        
	Kandace Lindsey                                                       MN0000855008        
	Candice Nelson                                                        MN0000940039        
	Candice Accola                                                        MN0000994639        
	Candace Stout                                                         MN0002535630

4670 / 4692   	 The Moving Sidewalks
Now Downloading: https://www.allmusic.com/search/artists/The%20Moving%20Sidewalks
	The Moving Sidewalks                                                  MN0000503563        
	Moving Hearts                                                         MN0000599262        
	Moving Units                                                          MN0000503947        
	Moving Targets                                                        MN0000503486        
	Moving Panoramas                                                      MN0003412136        
	Moving Mountains                                                      MN0000464784        
	Moving Pictures                                                       MN0000599359        
	Tiny Moving Parts                                                     MN0003282642        
	Chelsea Light Moving                                                  MN0002964700        
	The Moving Gelatine Plates                          

	The Alice Cooper Experience                                           MN0003552729        
	Alice Cooper Hall                                                     MN0003883104        
	Alice Capper                                                          MN0003043909        
	Alyssa Cooper                                                         MN0002082975        
	Alsey "Grub" Cooper                                                   MN0001264393        
	Bob Cooper                                                            MN0000761629        
	Wilma Lee Cooper                                                      MN0000955937        
	Stoney Cooper                                                         MN0000628262        
  --> 56430
4677 / 4692   	 Boyzone
Now Downloading: https://www.allmusic.com/search/artists/Boyzone
	Boyzone                                                               MN0000083194        
	Boyzone                                                               

4683 / 4692   	 The Levellers
Now Downloading: https://www.allmusic.com/search/artists/The%20Levellers
	The Levellers                                                         MN0000092414        
	Levellers 5                                                           MN0000822680        
	Levellers Mutineers                                                   MN0000909162        
	Bernard Lavilliers                                                    MN0000640838        
	Xenia Löffler                                                         MN0002218138        
	Bill Lefler                                                           MN0000542988        
	John Lefler                                                           MN0000977803        
	Reese LaFlare                                                         MN0003771745        
	Franz Loffler                                                         MN0001643816        
	Hans Loffler                                                        

4688 / 4692   	 Billy Gibbons
Now Downloading: https://www.allmusic.com/search/artists/Billy%20Gibbons
	Billy Gibbons                                                         MN0000067539        
	Billy Gibbons & the BFG's                                             MN0003432432        
  --> 56612
4689 / 4692   	 Daryl Hall & John Oates
Now Downloading: https://www.allmusic.com/search/artists/Daryl%20Hall%20%26%20John%20Oates
  --> 56612
4690 / 4692   	 Les Beatniks
Now Downloading: https://www.allmusic.com/search/artists/Les%20Beatniks
	Les King & His Beatniks                                               MN0003282482        
	The Beatniks                                                          MN0001193761        
	The Beatniks                                                          MN0000034778        
	Beatniks                                                              MN0000715700        
	The Beatniks                                                          MN0001869866      

In [33]:

savename = "collections/allMusicIDMap.p"
print("Saving {0} AllMusic IDs to {1}".format(len(allMusicIDMap), savename))
saveFile(idata=allMusicIDMap, ifile=savename, debug=True)

Ellie Goulding 	 MN0001596642 	 https://www.allmusic.com/artist/ellie-goulding-mn0001596642


In [52]:
allMusicIDMap['MN0000961234']

['Dave Matthews Band',
 'https://www.allmusic.com/artist/dave-matthews-band-mn0000961234']

In [53]:
amInfo  = allMusicIDMap['MN0000961234']
url     = "{0}/discography/all".format(amInfo[1])
data, response = getURLData(url, debug=True)

Now Downloading: https://www.allmusic.com/artist/dave-matthews-band-mn0000961234/discography/all


In [57]:
savename = "MN0000961234.p"
force = True
print("Saving {0} (force={1})".format(savename, force))
saveJoblib(data=data, filename=savename, compress=True)

Saving MN0000961234.p (force=True)
  --> This file is 27.2kB.


In [58]:
data = getFile(savename)

In [ ]:
url = "https://www.allmusic.com/artist/dave-matthews-band-mn0000961234/discography/all"

# All Music

In [ ]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')

In [ ]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')
artsAM = artistsAM(discAM)

In [ ]:
artsAM.searchAllMusicForArtist('Creed')

In [ ]:
artistRef = "https://www.allmusic.com/artist/creed-warner-mn0003293932"
artistRef = "{0}/discography".format(artistRef)
artistRef
#urllib.parse.urljoin(artistRef) #, "/discography")

In [7]:
#from discogsUtils import allmusicUtils
# Basic stuff
%load_ext autoreload
%autoreload
from discogsUtils import allmusicUtils

discUtils = allmusicUtils()
artistID = discUtils.getArtistID('https://www.allmusic.com/artist/bliss-mn0001514655', debug=True)
print("retval = ",artistID)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found ID 0001514655 from https://www.allmusic.com/artist/bliss-mn0001514655
retval =  0001514655


In [4]:
allMusicIDMap = {}
savename = "collections/allMusicIDMap.p"
allMusicIDMap = getFile(savename)
len(allMusicIDMap)

56646

In [11]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')
artsAM = artistsAM(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Warning! Saved Discog Directory None is not Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Users/tgadfort/Music/Discog/artists-allmusic exists
/Users/tgadfort/Music/Discog/albums-allmusic exists
/Users/tgadfort/Music/Discog/artists-allmusic-db exists
/Users/tgadfort/Music/Discog/albums-allmusic-db exists
/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata exists
/Users/tgadfort/Music/Discog/albums-allmusic-db/metadata exists
/Users/tgadfort/Music/Discog/diagnostic exists
/Users/tgadfort/Music/Discog/db exists


In [20]:
allMusicIDMap['MN0000986327']

['Billy Lee Myers, Jr.',
 'https://www.allmusic.com/artist/billy-lee-myers-jr-mn0000986327']

In [ ]:
from time import sleep
for i,(artistID,artistVals) in enumerate(allMusicIDMap.items()):
    if i <= 33396:
        continue
    artistName,ref = artistVals
    print(i,'/',len(allMusicIDMap),'\t',artistName,'\t',ref)
    url      = artsAM.getArtistURL(ref)
    savename = artsAM.getArtistSavename(artistID)
    artsAM.downloadArtistURL(url, savename, force=False)

33397 / 56646 	 Saxon & Cross 	 https://www.allmusic.com/artist/saxon-cross-mn0001492951
33398 / 56646 	 Saxon State Orchestra 	 https://www.allmusic.com/artist/saxon-state-orchestra-mn0001665155
Saving /Users/tgadfort/Music/Discog/artists-allmusic/55/0001665155.p (force=False)
  --> This file is 14.9kB.
Done. Sleeping for 2 seconds
33399 / 56646 	 Saxon Violence 	 https://www.allmusic.com/artist/saxon-violence-mn0000667456
Saving /Users/tgadfort/Music/Discog/artists-allmusic/56/0000667456.p (force=False)
  --> This file is 15.3kB.
Done. Sleeping for 2 seconds
33400 / 56646 	 Saxon Rowlings 	 https://www.allmusic.com/artist/saxon-rowlings-mn0001379768
Saving /Users/tgadfort/Music/Discog/artists-allmusic/68/0001379768.p (force=False)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
33401 / 56646 	 Saxon Christin 	 https://www.allmusic.com/artist/saxon-christin-mn0001623415
Saving /Users/tgadfort/Music/Discog/artists-allmusic/15/0001623415.p (force=False)
  --> This file is 15.2kB

33433 / 56646 	 Evan Schiller 	 https://www.allmusic.com/artist/evan-schiller-mn0000145657
Saving /Users/tgadfort/Music/Discog/artists-allmusic/57/0000145657.p (force=False)
  --> This file is 16.5kB.
Done. Sleeping for 2 seconds
33434 / 56646 	 Pat Schiller 	 https://www.allmusic.com/artist/pat-schiller-mn0000179791
Saving /Users/tgadfort/Music/Discog/artists-allmusic/91/0000179791.p (force=False)
  --> This file is 16.4kB.
Done. Sleeping for 2 seconds
33435 / 56646 	 Karl Schiller 	 https://www.allmusic.com/artist/karl-schiller-mn0000644196
Saving /Users/tgadfort/Music/Discog/artists-allmusic/96/0000644196.p (force=False)
  --> This file is 15.8kB.
Done. Sleeping for 2 seconds
33436 / 56646 	 Evan Schiller 	 https://www.allmusic.com/artist/evan-schiller-mn0000680762
Saving /Users/tgadfort/Music/Discog/artists-allmusic/62/0000680762.p (force=False)
  --> This file is 15.6kB.
Done. Sleeping for 2 seconds
33437 / 56646 	 Even Schiller 	 https://www.allmusic.com/artist/even-schiller-mn00

33469 / 56646 	 Schoolboy Cleve 	 https://www.allmusic.com/artist/schoolboy-cleve-mn0000314554
Saving /Users/tgadfort/Music/Discog/artists-allmusic/54/0000314554.p (force=False)
  --> This file is 16.2kB.
Done. Sleeping for 2 seconds
33470 / 56646 	 Schoolboy Duke 	 https://www.allmusic.com/artist/schoolboy-duke-mn0002534110
Saving /Users/tgadfort/Music/Discog/artists-allmusic/10/0002534110.p (force=False)
  --> This file is 15.8kB.
Done. Sleeping for 2 seconds
33471 / 56646 	 Schoolboy A 	 https://www.allmusic.com/artist/schoolboy-a-mn0003373573
Saving /Users/tgadfort/Music/Discog/artists-allmusic/73/0003373573.p (force=False)
  --> This file is 16.2kB.
Done. Sleeping for 2 seconds
33472 / 56646 	 Q Da Fool 	 https://www.allmusic.com/artist/q-da-fool-mn0003688281
Saving /Users/tgadfort/Music/Discog/artists-allmusic/81/0003688281.p (force=False)
  --> This file is 18.1kB.
Done. Sleeping for 2 seconds
33473 / 56646 	 Q;Indivi 	 https://www.allmusic.com/artist/q-indivi-mn0001976406
Savin

Saving /Users/tgadfort/Music/Discog/artists-allmusic/5/0000622505.p (force=False)
  --> This file is 17.6kB.
Done. Sleeping for 2 seconds
33505 / 56646 	 Annalisa 	 https://www.allmusic.com/artist/annalisa-mn0001216541
Saving /Users/tgadfort/Music/Discog/artists-allmusic/41/0001216541.p (force=False)
  --> This file is 19.9kB.
Done. Sleeping for 2 seconds
33506 / 56646 	 Sigrun Richter 	 https://www.allmusic.com/artist/sigrun-richter-mn0001668957
Saving /Users/tgadfort/Music/Discog/artists-allmusic/57/0001668957.p (force=False)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
33507 / 56646 	 Attention Screen 	 https://www.allmusic.com/artist/attention-screen-mn0002021703
Saving /Users/tgadfort/Music/Discog/artists-allmusic/3/0002021703.p (force=False)
  --> This file is 16.3kB.
Done. Sleeping for 2 seconds
33508 / 56646 	 Silver Screen 	 https://www.allmusic.com/artist/silver-screen-mn0002166996
Saving /Users/tgadfort/Music/Discog/artists-allmusic/96/0002166996.p (force=False)
 

33541 / 56646 	 Travis Scott 	 https://www.allmusic.com/artist/travis-scott-mn0002998422
Saving /Users/tgadfort/Music/Discog/artists-allmusic/22/0002998422.p (force=False)
  --> This file is 20.1kB.
Done. Sleeping for 2 seconds
33542 / 56646 	 Scott LaFaro 	 https://www.allmusic.com/artist/scott-lafaro-mn0000258090
Saving /Users/tgadfort/Music/Discog/artists-allmusic/90/0000258090.p (force=False)
  --> This file is 17.6kB.
Done. Sleeping for 2 seconds
33543 / 56646 	 Tom Scott 	 https://www.allmusic.com/artist/tom-scott-mn0000934350
Saving /Users/tgadfort/Music/Discog/artists-allmusic/50/0000934350.p (force=False)
  --> This file is 25.9kB.
Done. Sleeping for 2 seconds
33544 / 56646 	 Scott Henderson 	 https://www.allmusic.com/artist/scott-henderson-mn0000254833
Saving /Users/tgadfort/Music/Discog/artists-allmusic/33/0000254833.p (force=False)
  --> This file is 21.1kB.
Done. Sleeping for 2 seconds
33545 / 56646 	 Scott Hamilton 	 https://www.allmusic.com/artist/scott-hamilton-mn000025

...

Saving /Users/tgadfort/Music/Discog/artists-allmusic/77/0001787277.p (force=False)
  --> This file is 16.2kB.
Done. Sleeping for 2 seconds
34968 / 56646 	 Cooking Soul 	 https://www.allmusic.com/artist/cooking-soul-mn0003096549
Saving /Users/tgadfort/Music/Discog/artists-allmusic/49/0003096549.p (force=False)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
34969 / 56646 	 Soul Position 	 https://www.allmusic.com/artist/soul-position-mn0000040476
Saving /Users/tgadfort/Music/Discog/artists-allmusic/76/0000040476.p (force=False)
  --> This file is 18.0kB.
Done. Sleeping for 2 seconds
34970 / 56646 	 4 II Soul 	 https://www.allmusic.com/artist/4-ii-soul-mn0002003173
Saving /Users/tgadfort/Music/Discog/artists-allmusic/73/0002003173.p (force=False)
  --> This file is 15.8kB.
Done. Sleeping for 2 seconds
34971 / 56646 	 Soul to Soul 	 https://www.allmusic.com/artist/soul-to-soul-mn0000362531
Saving /Users/tgadfort/Music/Discog/artists-allmusic/31/0000362531.p (force=False)
  --> Thi

Saving /Users/tgadfort/Music/Discog/artists-allmusic/71/0000248871.p (force=False)
  --> This file is 16.4kB.
Done. Sleeping for 2 seconds
35004 / 56646 	 Salhatu Garba 	 https://www.allmusic.com/artist/salhatu-garba-mn0001870369
Saving /Users/tgadfort/Music/Discog/artists-allmusic/69/0001870369.p (force=False)
  --> This file is 14.3kB.
Done. Sleeping for 2 seconds
35005 / 56646 	 Salihede Erguner 	 https://www.allmusic.com/artist/salihede-erguner-mn0002301359
Saving /Users/tgadfort/Music/Discog/artists-allmusic/59/0002301359.p (force=False)
  --> This file is 14.7kB.
Done. Sleeping for 2 seconds
35006 / 56646 	 Silhouette Tales 	 https://www.allmusic.com/artist/silhouette-tales-mn0002992065
Saving /Users/tgadfort/Music/Discog/artists-allmusic/65/0002992065.p (force=False)
  --> This file is 15.7kB.
Done. Sleeping for 2 seconds
35007 / 56646 	 Sylhouette Musmin 	 https://www.allmusic.com/artist/sylhouette-musmin-mn0003187279
Saving /Users/tgadfort/Music/Discog/artists-allmusic/79/0003

Saving /Users/tgadfort/Music/Discog/artists-allmusic/66/0003547266.p (force=False)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
35039 / 56646 	 Dead Space Age 	 https://www.allmusic.com/artist/dead-space-age-mn0002843133
Saving /Users/tgadfort/Music/Discog/artists-allmusic/33/0002843133.p (force=False)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
35040 / 56646 	 Jaimyn Jupiter and the Space Age Rat Pak 	 https://www.allmusic.com/artist/jaimyn-jupiter-and-the-space-age-rat-pak-mn0003796079
Saving /Users/tgadfort/Music/Discog/artists-allmusic/79/0003796079.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
35041 / 56646 	 Age Of Space 	 https://www.allmusic.com/artist/age-of-space-mn0000522460
Saving /Users/tgadfort/Music/Discog/artists-allmusic/60/0000522460.p (force=False)
  --> This file is 15.5kB.
Done. Sleeping for 2 seconds
35042 / 56646 	 Responsible Space Playboys 	 https://www.allmusic.com/artist/responsible-space-playboys-mn0000439853


35073 / 56646 	 Generation of DeStiny 	 https://www.allmusic.com/artist/generation-of-destiny-mn0003236024
Saving /Users/tgadfort/Music/Discog/artists-allmusic/24/0003236024.p (force=False)
  --> This file is 15.8kB.
Done. Sleeping for 2 seconds
35074 / 56646 	 Branches of Destiny 	 https://www.allmusic.com/artist/branches-of-destiny-mn0003345055
Saving /Users/tgadfort/Music/Discog/artists-allmusic/55/0003345055.p (force=False)
  --> This file is 15.8kB.
Done. Sleeping for 2 seconds
35075 / 56646 	 Design of Destiny 	 https://www.allmusic.com/artist/design-of-destiny-mn0003352660
Saving /Users/tgadfort/Music/Discog/artists-allmusic/60/0003352660.p (force=False)
  --> This file is 15.7kB.
Done. Sleeping for 2 seconds
35076 / 56646 	 Gift of Destiny 	 https://www.allmusic.com/artist/gift-of-destiny-mn0003357631
Saving /Users/tgadfort/Music/Discog/artists-allmusic/31/0003357631.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
35077 / 56646 	 Special Skank 	 https://

Saving /Users/tgadfort/Music/Discog/artists-allmusic/23/0001977323.p (force=False)
  --> This file is 15.1kB.
Done. Sleeping for 2 seconds
35109 / 56646 	 Species 	 https://www.allmusic.com/artist/species-mn0002323271
Saving /Users/tgadfort/Music/Discog/artists-allmusic/71/0002323271.p (force=False)
  --> This file is 16.1kB.
Done. Sleeping for 2 seconds
35110 / 56646 	 S.P.E.C.I.E.S. 	 https://www.allmusic.com/artist/species-mn0002665992
Saving /Users/tgadfort/Music/Discog/artists-allmusic/92/0002665992.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
35111 / 56646 	 Species 	 https://www.allmusic.com/artist/species-mn0003251874
Saving /Users/tgadfort/Music/Discog/artists-allmusic/74/0003251874.p (force=False)
  --> This file is 15.8kB.
Done. Sleeping for 2 seconds
35112 / 56646 	 The Species 	 https://www.allmusic.com/artist/the-species-mn0003575854
Saving /Users/tgadfort/Music/Discog/artists-allmusic/54/0003575854.p (force=False)
  --> This file is 15.1kB.
Don

In [56]:
data = artsAM.getData('/Users/tgadfort/Music/Discog/artists-allmusic/62/0000567862.p')

In [17]:
file = "/Volumes/Music/Discog/artists-allmusic/68/0000782768.p"
file = "MN0000961234.p"
file = "/Users/tgadfort/Music/Discog/artists-allmusic/27/0000986327.p"
artAM = artistAM()
data = artAM.getData(file)

In [18]:
data

In [19]:
print("Artist")
info = data.artist.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("URL")
info = data.url.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("ID")
info = data.ID.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("Pages")
info = data.pages.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("Profile")
info = data.profile.get()
for k,v in info.items():
    if k == "profile":
        print("\t",k)
        for k2,v2 in v.items():
            print("\t\t",k2)
            for v3 in v2:
                for k4,v4 in v3.get().items():
                    if v4 is None:
                        print("\t\t\t",k4,'\t',v4)
                    elif isinstance(v4, str):
                        print("\t\t\t",k4,'\t',v4)
                    else:
                        print("\t\t\t",k4)
                        for k5,v5 in v4.get().items():
                            print("\t\t\t\t",k5,'\t',v5)
    elif k == "members":
        print("\t",k)
        for v2 in v:
            print("\t\t",v2)
    else:
        print('\t',k,'\t',v)
            

print("Media")
info = data.media.get()
for k,v in info.items():
    if k == "media":
        print("\t",k)
        for k2,v2 in v.items():
            print("\t\t",k2)
            for v3 in v2:
                for k4,v4 in v3.get().items():
                    if v4 is None:
                        print("\t\t\t",k4,'\t',v4)
                    elif isinstance(v4, str):
                        print("\t\t\t",k4,'\t',v4)
                    else:
                        print("\t\t\t",k4)
                        for k5,v5 in v4.get().items():
                            print("\t\t\t\t",k5,'\t',v5)


print("MediaCounts")
info = data.mediaCounts.get()
for k,v in info.items():
    print("\t",k,'\t',v)

Artist
	 name 	 None
	 err 	 True
URL
	 url 	 None
	 err 	 NoArtist
ID
	 ID 	 None
	 err 	 NotStr
Pages
	 ppp 	 1
	 tot 	 1
	 pages 	 1
	 err 	 None
	 more 	 False
	 redo 	 False
Profile
	 profile


AttributeError: 'NoneType' object has no attribute 'items'